In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import glob
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize

notebook_name = "cpol_demo.ipynb"
__file__ = "/home/ewan/Documents/THOR/examples/cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Parent directory for saving outputs
base_local = Path("/home/ewan/THOR_output")
start = "2005-11-13T13:00:00"
end = "2005-11-13T21:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
era5_pl_options = data.era5.data_options(start=start, end=end)
era5_sl_options = data.era5.data_options(
    start=start, end=end, data_format="single-levels"
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="cpol_era5")

grid_options = grid.create_options(name="geographic")
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol", tags=["era5_pl", "era5_sl"])
option.save_track_options(track_options, filename="cpol_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["cell", "anvil", "mcs"]
}
visualize_options["middle_cloud"] = visualize.option.runtime_options(
    "middle_cloud", save=True, style="presentation", figure_types=["mask"]
)
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-07-13 18:41:05,857 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/cpol_era5.yaml
2024-07-13 18:41:05,875 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-07-13 18:41:05,877 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/cpol_geographic.yaml
2024-07-13 18:41:05,886 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/tag_options/era5.yaml
2024-07-13 18:41:05,888 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/cpol_mcs.yaml
2024-07-13 18:41:05,898 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/visualize_options/runtime_mcs.yaml


In [44]:
output_directory = base_local / "runs/cpol_demo_geographic"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=output_directory
)

2024-07-12 23:53:41,535 - thor.track - INFO - Processing 2005-11-13T13:00:09.
2024-07-12 23:53:41,537 - thor.data.aura - DEBUG - Updating cpol dataset for 2005-11-13T13:00:09.
2024-07-12 23:53:41,537 - thor.data.aura - DEBUG - Converting cpol data from twp10cpolgrid150.b2.20051113.130000.nc
2024-07-12 23:53:41,586 - thor.data.aura - DEBUG - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-07-12 23:53:42,107 - thor.track - DEBUG - Processing hierarchy level 0.
2024-07-12 23:53:42,109 - thor.track - DEBUG - Tracking cell.
2024-07-12 23:53:42,127 - thor.match.match - DEBUG - Matching cell objects.
2024-07-12 23:53:42,131 - thor.match.match - DEBUG - No previous mask, or no objects in previous mask.
2024-07-12 23:53:42,285 - thor.visualize.runtime - DEBUG - Saving mask figure for cell.
2024-07-12 23:53:44,573 - thor.visualize.runtime - DEBUG - Saving match figure for cell.
2024-07-12 23:53:47,855 - thor.track - DEBUG - Tracking middle_cloud.
2024-07-12 23:53:48,006 - thor.v

# Cartesian Coordinates

In [3]:
grid_options = grid.create_options(name="cartesian", regrid=True)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_cartesian")

2024-07-13 18:41:05,987 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-07-13 18:41:05,990 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/cpol_cartesian.yaml


In [7]:
output_directory = base_local / "runs/cpol_cartesian_demo"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=base_local / "runs/cpol_cartesian_demo",
)

2024-07-13 18:48:57,611 - thor.track - INFO - Processing 2005-11-13T13:00:09.
2024-07-13 18:48:57,616 - thor.data.aura - DEBUG - Updating cpol dataset for 2005-11-13T13:00:09.
2024-07-13 18:48:57,619 - thor.data.aura - DEBUG - Converting cpol data from twp10cpolgrid150.b2.20051113.130000.nc


2024-07-13 18:48:57,956 - thor.track - DEBUG - Processing hierarchy level 0.
2024-07-13 18:48:57,957 - thor.track - DEBUG - Tracking cell.
2024-07-13 18:48:57,975 - thor.match.match - DEBUG - Matching cell objects.
2024-07-13 18:48:57,977 - thor.match.match - DEBUG - No previous mask, or no objects in previous mask.
2024-07-13 18:48:58,130 - thor.visualize.runtime - DEBUG - Saving mask figure for cell.
2024-07-13 18:48:59,580 - thor.visualize.runtime - DEBUG - Saving match figure for cell.
2024-07-13 18:49:02,762 - thor.track - DEBUG - Tracking middle_cloud.
2024-07-13 18:49:02,897 - thor.visualize.runtime - DEBUG - Saving mask figure for middle_cloud.
2024-07-13 18:49:04,074 - thor.track - DEBUG - Tracking anvil.
2024-07-13 18:49:04,079 - thor.match.match - DEBUG - Matching anvil objects.
2024-07-13 18:49:04,081 - thor.match.match - DEBUG - No previous mask, or no objects in previous mask.
2024-07-13 18:49:04,188 - thor.visualize.runtime - DEBUG - Saving mask figure for anvil.
2024-07